In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install efficientnet

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
import zipfile
import efficientnet.keras as efn   #efficientnet模型

In [ ]:
"""
弄到colab/content硬碟底下讀取速度比較快

"""
file_dir = '/content/drive/My Drive/shopee-product-detection-dataset.zip 的副本'  # 你的压缩包路径
zipFile = zipfile.ZipFile(file_dir)
for file in zipFile.namelist():
    zipFile.extract(file, '/content/shopee')  # 解压路径
zipFile.close()


In [ ]:
ROWS=600
COLS=600
batch_size = 32
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
train_idg = ImageDataGenerator(vertical_flip=False,
                               horizontal_flip=False,
                               height_shift_range=0.1,
                               width_shift_range=0.1,
                               validation_split=0.1, 
                               preprocessing_function=preprocess_input)
train_gen = train_idg.flow_from_directory(
    r"/content/shopee/train/train",
    target_size=(ROWS, COLS),
    batch_size = batch_size
)

In [ ]:
train_gen = train_idg.flow_from_directory(
    r"/content/shopee/train/train",
    target_size=(ROWS, COLS),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_gen = train_idg.flow_from_directory(
    r"/content/shopee/train/train",
    target_size=(ROWS, COLS),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')


In [ ]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from keras.utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import load_model


In [ ]:


Adam = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=False)
input_shape = (ROWS, COLS, 3)

nclass = len(train_gen.class_indices)

base_model = efn.EfficientNetB7(weights='imagenet', 
                                include_top=False, 
                                pooling='avg',
                                input_shape=(ROWS, COLS,3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
# add_model.add(Dense(600,activation='relu'))
# add_model.add(Dropout(0.2))
# add_model.add(Dense(300,activation='relu'))
# add_model.add(Dropout(0.2))
# add_model.add(Flatten())
add_model.add(Dense(nclass, 
                    activation='softmax'))
model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam,
              metrics=['accuracy'])
model.summary()

In [ ]:
"""
模型存成json檔
"""
model_json = model.to_json()
with open("/content/drive/My Drive/DL/model/EfficientNetB7.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
filepath="/content/drive/My Drive/DL/model/EfficientNetB7_weights.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
# mode='max')
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]



history = model.fit_generator(
    train_gen,
    steps_per_epoch = 3294,
    # validation_data = validation_gen, 
    # validation_steps = 1976,
    callbacks=callbacks_list,
    epochs = 10 )

In [ ]:
"""
model 開啟json檔
"""
from keras.models import model_from_json
model1 = model_from_json(open('/content/drive/My Drive/DL/model/EfficientNetB7.json').read())

"""
json模型載入權重
"""
model1.load_weights("/content/drive/My Drive/DL/model/EfficientNetB7_weights.h5")
model1.summary()


In [ ]:
model1.save('/content/drive/My Drive/DL/model/EfficientNetB7_new.h5')  

In [ ]:
"""
目的是要讓自己的層數可以訓練而不動到其他層

"""

model1.trainable = True   #改成可以訓練
model1.layers[0].trainable=False  #把
model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy', 
              optimizer="adam",
              metrics=['accuracy'])


In [ ]:
"""
怕colab自動登出,給他一行程式跑
"""

import time
a=0
while True:
    a+=1
    time.sleep(5)


In [ ]:
import shutil
shutil.copyfile("/content/drive/My Drive/DL/model/EfficientNetB7.h5", r"/content/EfficientNetB7.h5")

In [ ]:
!/opt/bin/nvidia-smi